# Bubble Generation Algorithm Experiments

This notebook allows you to experiment with different algorithms for the `calculate_bubbles` function from `generate_bubbles.py`.

## Setup and Imports

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, MultiPolygon, LineString
from shapely import union_all, minimum_rotated_rectangle, buffer, is_empty
from shapely.validation import make_valid
import time
import pandas as pd

from generate_bubbles import (
    get_boundaries,
    calculate_bubbles,
    calculate_radius_upper_bound, 
    calculate_step,
    compute_coverage_stats,
    BUBBLE_LIMIT
)

# Set up matplotlib for inline plotting
%matplotlib inline
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

## Load Sample Boundary Data

In [ ]:
USE_WARDS = False  # Set to True to use wards instead of constituencies

print("Loading boundary data...")
boundaries, output_type = get_boundaries(USE_WARDS)
print(f"Loaded {len(boundaries)} {output_type}")

# Display available boundaries
print("\nFirst 10 boundaries:")
for i, (name, _) in enumerate(boundaries[:10]):
    print(f"{i}: {name}")

## Select a Test Boundary

In [ ]:
# Select a boundary to experiment with
BOUNDARY_INDEX = 0  # Change this to select different boundaries
BOUNDARY_NAME = None  # Or set a specific name like "Birmingham, Edgbaston"

if BOUNDARY_NAME:
    # Find boundary by name
    test_boundary = None
    for name, boundary in boundaries:
        if name == BOUNDARY_NAME:
            test_boundary = (name, boundary)
            break
    if test_boundary is None:
        print(f"Boundary '{BOUNDARY_NAME}' not found!")
        test_boundary = boundaries[BOUNDARY_INDEX]
else:
    test_boundary = boundaries[BOUNDARY_INDEX]

boundary_name, boundary_geom = test_boundary
print(f"Selected boundary: {boundary_name}")
print(f"Area: {boundary_geom.area / 1_000_000:.2f} km²")
print(f"Max radius upper bound: {calculate_radius_upper_bound(boundary_geom)}m")

## Alternative Algorithm: Extend inclusion bubble area and add exclusion bubbles

In [4]:
def calculate_bubbles_with_exclusions(boundary):
    """
    Original bubble calculation algorithm from generate_bubbles.py
    """
    radius = calculate_radius_upper_bound(boundary)
    island_of_possibility = None

    inclusion_bubbles = []
    inclusion_data = []

    padded_boundary = boundary.buffer(500)

    while radius > 0 and len(inclusion_bubbles) < BUBBLE_LIMIT:
        island_of_possibility = buffer(padded_boundary, -(radius + 30))

        if not is_empty(island_of_possibility):
            print(f'   {radius}')
            polygons = island_of_possibility.geoms if isinstance(island_of_possibility, MultiPolygon) else [island_of_possibility]

            step = calculate_step(polygons, radius, len(inclusion_bubbles))
            for polygon in polygons:
                for interpolation in np.arange(0, polygon.exterior.length, step):
                    point = polygon.exterior.interpolate(interpolation)
                    bubble = point.buffer(radius)
                    if padded_boundary.contains(bubble):
                        inclusion_bubbles.append(bubble)
                        inclusion_data.append([point.x, point.y, int(radius / 1000)])

        if len(inclusion_bubbles) > 0:
            radius = (radius // 1500) * 1000
        else:
            radius -= 1000

    # Generate exclusion bubbles around the perimeter
    exclusion_bubbles = []
    exclusion_data = []
    
    # Use a smaller radius for exclusion bubbles
    exclusion_radius = 1000  # 1km radius for exclusion bubbles
    
    # Get the boundary exterior
    padded_boundary = boundary.buffer(1000)
    polygons = padded_boundary.geoms if isinstance(padded_boundary, MultiPolygon) else [padded_boundary]
    
    for polygon in polygons:
        if isinstance(polygon, LineString):
            continue
            
        # Calculate step size based on the perimeter length
        perimeter = polygon.exterior.length
        step = exclusion_radius / 4  # Some overlap
        
        # Place exclusion bubbles along the perimeter
        for distance in np.arange(0, perimeter, step):
            point = polygon.exterior.interpolate(distance)
            bubble = point.buffer(exclusion_radius)
            exclusion_bubbles.append(bubble)
            exclusion_data.append([point.x, point.y, int(exclusion_radius / 1000)])

    return (
        inclusion_bubbles[:BUBBLE_LIMIT],
        inclusion_data[:BUBBLE_LIMIT],
        exclusion_bubbles,
        exclusion_data
    )

## Alternative Algorithm: Grid-Based Approach

Let's try a different approach using a grid-based algorithm:

In [5]:
def calculate_bubbles_grid(boundary, grid_resolution=500):
    """
    Grid-based bubble placement algorithm
    
    Args:
        boundary: Shapely geometry
        grid_resolution: Distance between grid points in meters
    """
    inclusion_bubbles = []
    inclusion_data = []
    
    # Get boundary bounds
    minx, miny, maxx, maxy = boundary.bounds
    
    # Create grid points
    x_points = np.arange(minx, maxx, grid_resolution)
    y_points = np.arange(miny, maxy, grid_resolution)
    
    # Test different radii
    max_radius = calculate_radius_upper_bound(boundary)
    radii = range(max_radius, 500, -500)  # Decrease by 500m steps
    
    padded_boundary = boundary.buffer(500)
    
    for radius in radii:
        if len(inclusion_bubbles) >= BUBBLE_LIMIT:
            break
            
        print(f'   Grid approach - radius: {radius}m')
        
        for x in x_points:
            for y in y_points:
                if len(inclusion_bubbles) >= BUBBLE_LIMIT:
                    break
                    
                point = Point(x, y)
                if boundary.contains(point):
                    bubble = point.buffer(radius)
                    if padded_boundary.contains(bubble):
                        inclusion_bubbles.append(bubble)
                        inclusion_data.append([x, y, int(radius / 1000)])
        
        # If we got some bubbles with this radius, reduce grid resolution for next iteration
        if inclusion_bubbles:
            grid_resolution = grid_resolution * 0.7
    
    # Generate exclusion bubbles (same as original)
    exclusion_bubbles = []
    exclusion_data = []
    exclusion_radius = 1000
    
    padded_boundary = boundary.buffer(1000)
    polygons = padded_boundary.geoms if isinstance(padded_boundary, MultiPolygon) else [padded_boundary]
    
    for polygon in polygons:
        if isinstance(polygon, LineString):
            continue
        
        perimeter = polygon.exterior.length
        step = exclusion_radius / 4
        
        for distance in np.arange(0, perimeter, step):
            point = polygon.exterior.interpolate(distance)
            bubble = point.buffer(exclusion_radius)
            exclusion_bubbles.append(bubble)
            exclusion_data.append([point.x, point.y, int(exclusion_radius / 1000)])
    
    return (
        inclusion_bubbles[:BUBBLE_LIMIT],
        inclusion_data[:BUBBLE_LIMIT],
        exclusion_bubbles,
        exclusion_data
    )

## Alternative Algorithm: Random Sampling

A Monte Carlo approach with random sampling:

In [6]:
def calculate_bubbles_random(boundary, num_samples=10000, seed=42):
    """
    Random sampling bubble placement algorithm
    
    Args:
        boundary: Shapely geometry
        num_samples: Number of random points to sample
        seed: Random seed for reproducibility
    """
    np.random.seed(seed)
    
    inclusion_bubbles = []
    inclusion_data = []
    
    # Get boundary bounds
    minx, miny, maxx, maxy = boundary.bounds
    
    # Generate random points within bounding box
    x_coords = np.random.uniform(minx, maxx, num_samples)
    y_coords = np.random.uniform(miny, maxy, num_samples)
    
    # Test different radii
    max_radius = calculate_radius_upper_bound(boundary)
    radii = range(max_radius, 500, -1000)  # Decrease by 1000m steps
    
    padded_boundary = boundary.buffer(500)
    
    for radius in radii:
        if len(inclusion_bubbles) >= BUBBLE_LIMIT:
            break
            
        print(f'   Random approach - radius: {radius}m')
        
        for x, y in zip(x_coords, y_coords):
            if len(inclusion_bubbles) >= BUBBLE_LIMIT:
                break
                
            point = Point(x, y)
            if boundary.contains(point):
                bubble = point.buffer(radius)
                if padded_boundary.contains(bubble):
                    # Check for significant overlap with existing bubbles
                    overlap = False
                    for existing_bubble in inclusion_bubbles:
                        if bubble.intersection(existing_bubble).area > 0.3 * bubble.area:
                            overlap = True
                            break
                    
                    if not overlap:
                        inclusion_bubbles.append(bubble)
                        inclusion_data.append([x, y, int(radius / 1000)])
    
    # Generate exclusion bubbles (same as original)
    exclusion_bubbles = []
    exclusion_data = []
    exclusion_radius = 1000
    
    padded_boundary = boundary.buffer(1000)
    polygons = padded_boundary.geoms if isinstance(padded_boundary, MultiPolygon) else [padded_boundary]
    
    for polygon in polygons:
        if isinstance(polygon, LineString):
            continue
        
        perimeter = polygon.exterior.length
        step = exclusion_radius / 4
        
        for distance in np.arange(0, perimeter, step):
            point = polygon.exterior.interpolate(distance)
            bubble = point.buffer(exclusion_radius)
            exclusion_bubbles.append(bubble)
            exclusion_data.append([point.x, point.y, int(exclusion_radius / 1000)])
    
    return (
        inclusion_bubbles[:BUBBLE_LIMIT],
        inclusion_data[:BUBBLE_LIMIT],
        exclusion_bubbles,
        exclusion_data
    )

## Comparison Function

Function to compare different algorithms:

In [7]:
def compare_algorithms(boundary, algorithms):
    """
    Compare different bubble generation algorithms
    
    Args:
        boundary: Shapely geometry to test on
        algorithms: Dict of {name: function} pairs
    
    Returns:
        Dict with results for each algorithm
    """
    results = {}
    
    for name, algorithm in algorithms.items():
        print(f"\n--- Running {name} ---")
        start_time = time.time()
        
        try:
            inclusion_bubbles, inclusion_data, exclusion_bubbles, exclusion_data = algorithm(boundary)
            coverage_stats = compute_coverage_stats(boundary, inclusion_bubbles, exclusion_bubbles)
            
            inclusion_coverage = coverage_stats["internal_inclusion"]
            exclusion_coverage = coverage_stats["exclusion"]
            external_inclusion_coverage = coverage_stats["external_inclusion"]
            net_coverage = coverage_stats["net"]

            results[name] = {
                'inclusion_bubbles': inclusion_bubbles,
                'inclusion_data': inclusion_data,
                'exclusion_bubbles': exclusion_bubbles,
                'exclusion_data': exclusion_data,
                'inclusion_coverage': inclusion_coverage,
                'exclusion_coverage': exclusion_coverage,
                'external_inclusion_coverage': external_inclusion_coverage,
                'net_coverage': net_coverage,
                'num_inclusion_bubbles': len(inclusion_bubbles),
                'num_exclusion_bubbles': len(exclusion_bubbles),
                'execution_time': time.time() - start_time,
                'success': True
            }
            
            print(f"   ✓ Success: {len(inclusion_bubbles)} inclusion, {len(exclusion_bubbles)} exclusion bubbles")
            print(f"   ✓ Coverage: {inclusion_coverage:.1f}% inclusion, {net_coverage:.1f}% net")
            print(f"   ✓ Time: {time.time() - start_time:.2f}s")
            
        except Exception as e:
            print(f"   ✗ Failed: {e}")
            results[name] = {
                'success': False,
                'error': str(e),
                'execution_time': time.time() - start_time
            }
    
    return results

## Run Comparison

Now let's test all algorithms on our selected boundary:

In [ ]:
# Define algorithms to compare
algorithms = {
    'Original': calculate_bubbles,
    'With Exclusions': calculate_bubbles_with_exclusions,
    'Grid-based': calculate_bubbles_grid,
    'Random sampling': calculate_bubbles_random
}

# Run comparison
print(f"Testing algorithms on: {boundary_name}")
results = compare_algorithms(boundary_geom, algorithms)

# Create summary table
summary_data = []
for name, result in results.items():
    if result['success']:
        summary_data.append({
            'Algorithm': name,
            'Inclusion Bubbles': result['num_inclusion_bubbles'],
            'Exclusion Bubbles': result['num_exclusion_bubbles'],
            'Inclusion Coverage (%)': f"{result['inclusion_coverage']:.1f}",
            'Net Coverage (%)': f"{result['net_coverage']:.1f}",
            'External Inclusion Coverage (%)': f"{result['external_inclusion_coverage']:.1f}",
            'Exclusion Coverage (%)': f"{result['exclusion_coverage']:.1f}",
            'Execution Time (s)': f"{result['execution_time']:.2f}"
        })
    else:
        summary_data.append({
            'Algorithm': name,
            'Inclusion Bubbles': 'FAILED',
            'Exclusion Bubbles': 'FAILED',
            'Inclusion Coverage (%)': 'FAILED',
            'Net Coverage (%)': 'FAILED',
            'Execution Time (s)': f"{result['execution_time']:.2f}"
        })

summary_df = pd.DataFrame(summary_data)
print("\n=== SUMMARY ===")
print(summary_df.to_string(index=False))

## Visualization Comparison

Let's visualize the results from different algorithms:

In [ ]:
def plot_algorithm_comparison(boundary, results, boundary_name):
    """
    Create a comparison plot showing results from different algorithms
    """
    successful_results = {name: result for name, result in results.items() if result['success']}
    
    if not successful_results:
        print("No successful results to plot")
        return
    
    n_algorithms = len(successful_results)
    fig, axes = plt.subplots(2, n_algorithms, figsize=(5*n_algorithms, 10))
    
    if n_algorithms == 1:
        axes = axes.reshape(2, 1)
    
    fig.suptitle(f'Algorithm Comparison: {boundary_name}', fontsize=16)
    
    for i, (name, result) in enumerate(successful_results.items()):
        # Top row: outline view
        ax1 = axes[0, i]
        ax1.set_title(f'{name}\n(Outline View)')
        ax1.set_aspect('equal')
        
        # Plot boundary
        if hasattr(boundary, 'geoms'):
            polygons = boundary.geoms
        else:
            polygons = [boundary]
            
        for polygon in polygons:
            if hasattr(polygon, 'exterior'):
                x, y = polygon.exterior.xy
                ax1.plot(x, y, color='blue', linewidth=2)
        
        # Plot inclusion bubbles
        for bubble in result['inclusion_bubbles']:
            x, y = bubble.exterior.xy
            ax1.plot(x, y, color='green', linewidth=0.5, alpha=0.7)
        
        # Plot exclusion bubbles
        for bubble in result['exclusion_bubbles']:
            x, y = bubble.exterior.xy
            ax1.plot(x, y, color='red', linewidth=0.5, alpha=0.7)
        
        ax1.set_xticks([])
        ax1.set_yticks([])
        
        # Bottom row: filled view
        ax2 = axes[1, i]
        ax2.set_title(f'Net Coverage: {result["net_coverage"]:.1f}%\n({result["num_inclusion_bubbles"]} bubbles)')
        ax2.set_aspect('equal')
        
        # Plot boundary
        for polygon in polygons:
            if hasattr(polygon, 'exterior'):
                x, y = polygon.exterior.xy
                ax2.plot(x, y, color='blue', linewidth=2)
        
        # Plot filled inclusion bubbles
        for bubble in result['inclusion_bubbles']:
            x, y = bubble.exterior.xy
            ax2.fill(x, y, color='green', alpha=0.3)
        
        # Plot filled exclusion bubbles
        for bubble in result['exclusion_bubbles']:
            x, y = bubble.exterior.xy
            ax2.fill(x, y, color='red', alpha=0.3)
        
        ax2.set_xticks([])
        ax2.set_yticks([])
    
    plt.tight_layout()
    plt.show()

# Create the comparison plot
plot_algorithm_comparison(boundary_geom, results, boundary_name)

## Custom Algorithm Development

Use this cell to develop and test your own algorithm:

In [ ]:
def calculate_bubbles_custom(boundary):
    """
    Your custom bubble generation algorithm
    
    Modify this function to implement your own approach!
    
    Args:
        boundary: Shapely geometry object
    
    Returns:
        tuple: (inclusion_bubbles, inclusion_data, exclusion_bubbles, exclusion_data)
    """
    # TODO: Implement your custom algorithm here
    
    # Example: Start with the original algorithm and modify it
    return calculate_bubbles_original(boundary)

# Test your custom algorithm
print("Testing custom algorithm...")
custom_results = compare_algorithms(boundary_geom, {'Custom': calculate_bubbles_custom})

if custom_results['Custom']['success']:
    print(f"Custom algorithm coverage: {custom_results['Custom']['net_coverage']:.1f}%")
    plot_algorithm_comparison(boundary_geom, custom_results, f"{boundary_name} - Custom Algorithm")

## Batch Testing

Test algorithms on multiple boundaries:

In [11]:
def batch_test_algorithms(boundaries, algorithms, max_boundaries=5):
    """
    Test algorithms on multiple boundaries
    """
    test_boundaries = boundaries[:max_boundaries]
    batch_results = []
    
    for i, (name, boundary) in enumerate(test_boundaries):
        print(f"\n{'='*50}")
        print(f"Testing boundary {i+1}/{len(test_boundaries)}: {name}")
        print(f"{'='*50}")
        
        boundary_results = compare_algorithms(boundary, algorithms)
        
        for alg_name, result in boundary_results.items():
            if result['success']:
                batch_results.append({
                    'Boundary': name,
                    'Algorithm': alg_name,
                    'Net Coverage (%)': result['net_coverage'],
                    'Inclusion Bubbles': result['num_inclusion_bubbles'],
                    'Execution Time (s)': result['execution_time']
                })
    
    return pd.DataFrame(batch_results)

# Run batch test (uncomment to run)
# batch_df = batch_test_algorithms(boundaries, algorithms, max_boundaries=3)
# print("\n=== BATCH TEST RESULTS ===")
# print(batch_df.to_string(index=False))

# # Calculate average performance by algorithm
# avg_performance = batch_df.groupby('Algorithm').agg({
#     'Net Coverage (%)': 'mean',
#     'Inclusion Bubbles': 'mean',
#     'Execution Time (s)': 'mean'
# }).round(2)
# print("\n=== AVERAGE PERFORMANCE ===")
# print(avg_performance)

## Utility Functions

Additional helper functions for algorithm development:

In [ ]:
def analyze_boundary(boundary, name):
    """
    Analyze characteristics of a boundary that might inform algorithm design
    """
    area_km2 = boundary.area / 1_000_000
    perimeter_km = boundary.length / 1000
    
    # Calculate shape complexity (perimeter^2 / area)
    complexity = (boundary.length ** 2) / boundary.area
    
    # Get bounding box dimensions
    minx, miny, maxx, maxy = boundary.bounds
    width_km = (maxx - minx) / 1000
    height_km = (maxy - miny) / 1000
    
    print(f"Boundary Analysis: {name}")
    print(f"  Area: {area_km2:.2f} km²")
    print(f"  Perimeter: {perimeter_km:.2f} km")
    print(f"  Bounding box: {width_km:.2f} × {height_km:.2f} km")
    print(f"  Shape complexity: {complexity:.2f}")
    print(f"  Max radius bound: {calculate_radius_upper_bound(boundary)}m")
    
    return {
        'area_km2': area_km2,
        'perimeter_km': perimeter_km,
        'width_km': width_km,
        'height_km': height_km,
        'complexity': complexity,
        'max_radius': calculate_radius_upper_bound(boundary)
    }

# Analyze current boundary
boundary_analysis = analyze_boundary(boundary_geom, boundary_name)